# This script demonstrates how to optimize a variational autoencoder with keras-tuner

In [28]:
# VAE
import vae_tools # Always import first to define if keras or tf.kreas should be used
import vae_tools.sanity
import vae_tools.sampling
vae_tools.sampling.set_seed(0)
from vae_tools.mmvae import MmVae, ReconstructionLoss
vae_tools.sanity.check()

# DL
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.datasets import mnist

# Hypertuning
from kerastuner.tuners import RandomSearch, Hyperband
import kerastuner as kt
from kerastuner import HyperModel

# Math
import numpy as np

python version:  3.5.2
keras version: 2.2.4-tf
tensorflow version: 2.0.2
matplotlib uses:  module://ipykernel.pylab.backend_inline
No GPUs available


In [29]:
# Get the MNIST digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Cut down data set
x_train = x_train[:10,:]
y_train = y_train[:10]
x_test = x_test[:10,:]
y_test = y_test[:10]

In [30]:
# Define the network and parameter structure

def lr_normalizer(lr, optimizer):
    """Assuming a default learning rate 1, rescales the learning rate
    such that learning rates amongst different optimizers are more or less
    equivalent.
    Parameters
    ----------
    lr : float
        The learning rate.
    optimizer : keras optimizer
        The optimizer. For example, Adagrad, Adam, RMSprop.
    """

    from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Adagrad, Adamax, RMSprop
    from tensorflow.keras.optimizers import Nadam

    if optimizer == Adadelta:
        pass
    elif optimizer == SGD or optimizer == Adagrad:
        lr /= 100.0
    elif optimizer == Adam or optimizer == RMSprop:
        lr /= 1000.0
    elif optimizer == Adamax or optimizer == Nadam:
        lr /= 500.0
    else:
        raise ValueError(str(optimizer) + " is not supported by lr_normalizer")
    return lr

img_rows, img_cols, img_chns = 28, 28, 1
original_dim = img_rows * img_cols * img_chns
class VaeHyperModel(HyperModel):

    def __init__(self, optimizer):
        super().__init__()
        self.optimizer = optimizer

    def build(self, hp):
        self.p = {'lr': hp.Fixed('lr', 1.),
             'intermediate_dim': hp.Choice('intermediate_dim', [64, 128, 256], ordered = True),
             'activation': hp.Choice('activation', ['relu', 'elu'], ordered = False),
             'batch_size': hp.Int('batch_size', 1024, 2048, 1024),
             'epochs': hp.Int('epochs', 10, 20, 10),
             'beta': hp.Choice('beta', [0.01, 1.0], ordered = True),
             'reconstruction_loss_metrics': hp.Choice('reconstruction_loss_metrics', [ReconstructionLoss.MSE]),
             'z_dim': hp.Fixed('z_dim', 2)}
        #vae_tools.sampling.set_seed(0)
        encoder = [[
            Input(shape=(original_dim,)),
            Dense(self.p['intermediate_dim'], activation=self.p['activation']),
            Dense(int(self.p['intermediate_dim']/2), activation=self.p['activation'])
        ]]

        decoder = [[
            Dense(int(self.p['intermediate_dim']/2), activation=self.p['activation']),
            Dense(self.p['intermediate_dim'], activation=self.p['activation']),
            Dense(original_dim, activation='sigmoid')
        ]]

        vae_obj = MmVae(self.p['z_dim'], encoder, decoder, [original_dim], self.p['beta'],
                        reconstruction_loss_metrics = [self.p['reconstruction_loss_metrics']], name='Vae')

        vae_model = vae_obj.get_model()
        vae_model.compile(optimizer=self.optimizer(lr_normalizer(self.p['lr'], self.optimizer)), loss=None)
        return vae_model


In [31]:
hp = kt.HyperParameters()

hypermodel = VaeHyperModel(optimizer = Adam)
# Hyperband, RandomSearch
tuner = Hyperband(
    hypermodel,
    objective='loss',
    #max_trials=20,
    max_epochs=20,
    directory='my_dir',
    project_name='helloworld')

# Parameter search
tuner.search([x_train, y_train], None,
             epochs=2,
             validation_data=([x_test, y_test], None))


Train on 10 samples, validate on 10 samples
Epoch 1/3
10/10 [==============================] - 1s 100ms/sample - loss: 543.6970 - loss_reconstruction_0_0: 543.6954 - loss_prior_0: 0.0016 - val_loss: 542.3751 - val_loss_reconstruction_0_0: 542.3732 - val_loss_prior_0: 0.0019
Epoch 2/3
10/10 [==============================] - 0s 4ms/sample - loss: 542.3937 - loss_reconstruction_0_0: 542.3921 - loss_prior_0: 0.0017 - val_loss: 540.7828 - val_loss_reconstruction_0_0: 540.7810 - val_loss_prior_0: 0.0018
Epoch 3/3
10/10 [==============================] - 0s 4ms/sample - loss: 540.9553 - loss_reconstruction_0_0: 540.9534 - loss_prior_0: 0.0019 - val_loss: 539.5323 - val_loss_reconstruction_0_0: 539.5306 - val_loss_prior_0: 0.0017
Train on 10 samples, validate on 10 samples
Epoch 1/3
10/10 [==============================] - 1s 89ms/sample - loss: 544.4296 - loss_reconstruction_0_0: 544.4261 - loss_prior_0: 0.0035 - val_loss: 545.3251 - val_loss_reconstruction_0_0: 545.3203 - val_loss_prior_0: 

In [38]:
#tuner.results_summary()
best_model= tuner.get_best_models(num_models=1)
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)

['Boolean', 'Choice', 'Fixed', 'Float', 'Int', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_name_is_valid', '_conditions_are_active', '_get_name', '_get_name_parts', '_is_conditional_scope', '_is_name_scope', '_retrieve', '_retrieve_helper', '_scopes', '_space', 'conditional_scope', 'copy', 'from_config', 'from_proto', 'get', 'get_config', 'merge', 'name_scope', 'register', 'space', 'to_proto', 'values']
{'z_dim': 2, 'reconstruction_loss_metrics': 'MSE', 'intermediate_dim': 256, 'tuner/initial_epoch': 0, 'tuner/round': 0, 'beta': 1.0, 'activation': 'elu', 'epochs': 20, 'lr': 1.0, 'tuner/bracket': 0, 'tuner/epochs': 20, 'batch_size': 1024}
